![head.png](https://github.com/iwh-halle/FinancialDataAnalytics/blob/master/figures/head.jpg?raw=1)

# Financial Data Analytics in Python

**Prof. Dr. Fabian Woebbeking**</br>
Assistant Professor of Financial Economics

IWH - Leibniz Institute for Economic Research</br>
MLU - Martin Luther University Halle-Wittenberg

fabian.woebbeking@iwh-halle.de

# Homework

You will need a Git/GitHub repository to submit your course deliverables. Consult [**slides.ipynb**](https://github.com/iwh-halle/FinancialDataAnalytics) for help with the tasks below! If you need further assistance, do not hesitate to open a Q&A at https://github.com/cafawo/FinancialDataAnalytics/discussions

### Task: 

The liquidity position of a firm measured in million is a **generalized Wiener process** with a drift of $0.1$ per month and a variance of $\sigma^2 = 0.16$ per month. The initial cash position is $2.0$. Calculate:

1. 	the expected value and standard deviation in one, six and 12 months.
2.	What is the probability of a negative cash position in 6 and 12 months? 

In [47]:
import numpy as np
import scipy.stats as stats
np.random.seed(888) # setting a seed for reproducibility

def calc_gen_wien_pro_X(drift, variance, init_cash, months):
    '''
    Calculates the general wealth with proportional return using the Monte Carlo method.

    Parameters:
    - drift (float): The expected average return per month.
    - variance (float): The variance of the return per month.
    - init_cash (float): The initial amount of cash.
    - months (int): The number of months to simulate.

    Returns:
    - wealth (float): The final wealth after the simulation.
    
    '''
    return init_cash + drift * months + (variance ** 0.5) * (months ** 0.5) * np.random.normal(0, 1)

def expected_value(drift, init_cash, months):
    """
    Calculates the expected value of an investment after a given number of months.
    
    Parameters:
    - drift (float): The average monthly return rate of the investment.
    - init_cash (float): The initial amount of cash invested.
    - months (int): The number of months the investment is held for.
    
    Returns:
    - float: The expected value of the investment after the given number of months.
    """
    return init_cash + drift * months

def standard_deviation(variance, months):
    """
    Calculate the standard deviation of a financial data series.
    
    Parameters:
    - variance (float): The variance of the data series.
    - months (int): The number of months over which the data is collected.
    
    Returns:
    - float: The standard deviation of the data series.
    """
    return (variance ** 0.5) * (months ** 0.5)

def probability_negative_cash(drift, variance, init_cash, months):
    """
    Calculates the probability of having negative cash at the end of the given time period.
    
    Parameters:
    - drift (float): The drift rate of the asset.
    - variance (float): The variance of the asset.
    - init_cash (float): The initial cash amount.
    - months (int): The number of months for which the probability is calculated.
    
    Returns:
    - probability (float): The probability of having negative cash at the end of the time period.
    """
    mean = expected_value(drift, init_cash, months)
    std_dev = standard_deviation(variance, months)
    return stats.norm.cdf(0, loc=mean, scale=std_dev)


def set_param_and_calc(drift, variance, init_cash, months):
    """
    Calculates and prints various financial metrics for a given set of parameters.
    
    Parameters:
    - drift (float): The drift rate of the financial asset.
    - variance (float): The variance of the financial asset.
    - init_cash (float): The initial cash value.
    - months (int): The number of months for which to calculate the metrics.
    
    Returns:
    Printing the calculated values.
    """
    if months == 1:
        print(f"For {months} month the calculated value of X(t) of the generalized wiener process is: {calc_gen_wien_pro_X(drift, variance, init_cash, months):.3f}")
        print(f"For {months} month the expected value is:{expected_value(drift, init_cash, months):.3f}")
        print(f"For {months} month the standard deviation is:{standard_deviation(variance, months):.3f}")
        
    
    else:
        print(f"For {months} months the calculated value of X(t) of the generalized wiener process is: {calc_gen_wien_pro_X(drift, variance, init_cash, months):.3f}")
        print(f"For {months} months the expected value is:{expected_value(drift, init_cash, months):.3f}")
        print(f"For {months} months the standard deviation is:{standard_deviation(variance, months):.3f}")
        print(f"For {months} months the probability of negative cash is:{probability_negative_cash(drift, variance, init_cash, months)*100:.3f}%")

    return

months = [1, 6, 12]

for month in months:
    set_param_and_calc(0.1, 0.16, 2, month)
    print("")


For 1 month the calculated value of X(t) of the generalized wiener process is: 2.030
For 1 month the expected value is:2.100
For 1 month the standard deviation is:0.400

For 6 months the calculated value of X(t) of the generalized wiener process is: 2.785
For 6 months the expected value is:2.600
For 6 months the standard deviation is:0.980
For 6 months the probability of negative cash is:0.398%

For 12 months the calculated value of X(t) of the generalized wiener process is: 4.346
For 12 months the expected value is:3.200
For 12 months the standard deviation is:1.386
For 12 months the probability of negative cash is:1.046%



### Task: 

The cash flow of a [call option](https://en.wikipedia.org/wiki/Call_option) with strike $K$ at maturity $T$ is given by

$$
max(S_T - K, 0) = (S_T - K)^+
$$

where $S_T$ is the price of the underlying at $T$. The price of the option under the [risk-neutral measure](https://en.wikipedia.org/wiki/Risk-neutral_measure) $\mathbb{Q}$ is simply its discounted expected value
$$
\mathbb{E}^\mathbb{Q}[(S_T - K)^+] e^{-rT}.
$$


Calculate the price of the option, using:
1. numerical integration and
2. Monte carlo simulation.

For you calculations, assume that todays price of the underlying is $S_0 = 220$, the strike is $K = 220$, volatility is $\sigma = 0.98$, the risk free rate is $r = 10\%$ (continuous) and maturity is one year. We further assume that the underlying $S$ follows a **Geometric Brownian motion**.

Hint: The terminal stock price $S_T$, under the risk-neutral measure, follows a log-normal distribution with PDF

$$f(x) = \frac{1}{x s \sqrt{2 \pi}} \exp\left( -\frac{(\ln x - \mu)^2}{2 s^2} \right) $$

where $\mu = \ln S_0 + (r-\sigma^2 / 2)T$ and variance $s^2 = \sigma^2 T$.


In [1]:
import numpy as np
from scipy.integrate import quad

# Settings and pre Calculations

# set Parameters
todays_price = 220      # S_0
strike_price = 220      # K
volatility = 0.98       # σ
risk_free_rate = 0.1    # r
maturity = 1            # T in years

# Calculate variance and standard deviation 
variance = volatility ** 2 * maturity
standard_deviation = variance ** 0.5

# Calculate mu 
mu = np.log(todays_price) + (risk_free_rate - 0.5 * volatility**2) * maturity


In [2]:
# Numerical Solution
# Define the function f(x) => x = S_T
def f(x):
    return 1 / (x*standard_deviation*(2*np.pi)**0.5) * np.exp(-(np.log(x)-mu)**2/(2 * standard_deviation**2))

# Define the function of the payoff of the call option x = S_T
def payoff(x):
    return max(x - strike_price, 0)

# Calculate the expected value of the payoff
expected_payoff, err = quad(lambda x: f(x) * payoff(x), 0, np.inf)

# Calculate the price of the call option
price = np.exp(-risk_free_rate * maturity) * expected_payoff

print(f"The expected payoff of the call option is: {expected_payoff}")
print(f"The price of the call option is: {price}")



The expected payoff of the call option is: 99.02488639513986
The price of the call option is: 89.60142252708256


In [6]:
#  Monte Carlo Simulation
np.random.seed(888) # setting a seed for reproducibility
num_simulations = 1000000

# Generate random samples from the log-normal distribution
ST = np.random.lognormal(mean=mu, sigma=np.sqrt(variance), size=num_simulations)

# Calculate the payoff of the call option for each simulation
payoff = np.maximum(ST - strike_price, 0)

# Calculate the average payoff and discount it
option_price = np.mean(payoff) * np.exp(-risk_free_rate * maturity)

print(f"The price of the call option using Monte Carlo simulation is: {option_price}")

The price of the call option using Monte Carlo simulation is: 89.87492741667802
